# Start

In [40]:
source_folder = "../test/"

loan_details_file = f"{source_folder}loan_details.xlsx"
geo_file = f"{source_folder}geographic_data.xml"
financial_file = f"{source_folder}financial_data.jsonl"
demographic_file = f"{source_folder}demographics.csv"
history_file = f"{source_folder}credit_history.parquet"
app_file = f"{source_folder}application_metadata.csv"

output_file = "../results.csv"

In [4]:
import pandas as pd

In [6]:
loan_details=pd.read_excel(loan_details_file)
loan_details['loan_amount']=loan_details['loan_amount'].replace('[\$,]', '', regex=True).astype(float)
loan_details.head()

,customer_id,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign
0,100000,Personal,11300.0,24,10.26,Debt Consolidation,0.000,Online,1066,T
1,100001,CC,6800.0,0,17.74,Revolving Credit,0.000,Branch,1041,B
2,100002,credit card,3400.0,0,19.42,Revolving Credit,0.000,Online,1001,W
3,100003,CC,12000.0,0,20.95,Revolving Credit,0.000,Online,1098,D
4,100004,MORTGAGE,279900.0,360,6.99,Refinance,0.798,Online,1067,H


In [94]:
loan_details.isna().sum()

customer_id            0
loan_type              0
loan_amount            0
loan_term              0
interest_rate          0
loan_purpose           0
loan_to_value_ratio    0
origination_channel    0
loan_officer_id        0
marketing_campaign     0
dtype: int64

In [95]:
loan_map = {
    'cc': 'Credit Card',
    'credit card': 'Credit Card',
    'creditcard': 'Credit Card',
    'home loan': 'Mortgage',
    'mortgage': 'Mortgage',
    'personal': 'Personal Loan',
    'personal loan': 'Personal Loan'
}
loan_details['loan_type'] = loan_details['loan_type'].str.lower().map(lambda x: loan_map.get(x, x))
print(loan_details)

       customer_id      loan_type  loan_amount  loan_term  interest_rate  \
0            10000  Personal Loan      17700.0         36          12.50   
1            10001       Mortgage     114000.0        180           6.83   
2            10002  Personal Loan       9300.0         36          13.99   
3            10003  Personal Loan       8700.0         48          13.26   
4            10004  Personal Loan       7200.0         24          10.77   
...            ...            ...          ...        ...            ...   
89994        99994  Personal Loan      11300.0         24           9.70   
89995        99995  Personal Loan      16500.0         36          11.67   
89996        99996  Personal Loan      17800.0         36          14.99   
89997        99997    Credit Card      11700.0          0          19.10   
89998        99998       Mortgage     487100.0        360           5.03   

             loan_purpose  loan_to_value_ratio origination_channel  \
0      Debt Conso

In [42]:
demografics=pd.read_csv(demographic_file)
demografics['annual_income']=demografics['annual_income'].replace('[\$,]', '', regex=True).astype(float)
demografics.head()

,cust_id,age,annual_income,employment_length,employment_type,education,marital_status,num_dependents
0,100000,57,52300.0,11.7,Self Employed,Some College,Divorced,0
1,100001,41,45700.0,5.5,SELF_EMPLOYED,High School,Married,1
2,100002,22,20000.0,4.4,Part Time,Graduate,Single,1
3,100003,27,44300.0,8.0,FULL_TIME,High School,Single,1
4,100004,32,41100.0,0.9,FT,Bachelor,Single,3


In [97]:
demografics.isna().sum()

cust_id                 0
age                     0
annual_income           0
employment_length    2253
employment_type         0
education               0
marital_status          0
num_dependents          0
dtype: int64

In [98]:
demografics['employment_length'] = demografics['employment_length'].fillna(0)

In [99]:
demografics.isna().sum()

cust_id              0
age                  0
annual_income        0
employment_length    0
employment_type      0
education            0
marital_status       0
num_dependents       0
dtype: int64

In [100]:
standardized_job_types = {
    'contract': 'contract',
    'contractor': 'contract',
    'ft': 'full_time',
    'full_time': 'full_time',
    'full time': 'full_time',
    'full-time': 'full_time',
    'fulltime': 'full_time',
    'pt': 'part_time',
    'part_time': 'part_time',
    'part time': 'part_time',
    'part-time': 'part_time',
    'self_employed': 'self_employed',
    'self emp': 'self_employed',
    'self employed': 'self_employed',
    'self-employed': 'self_employed'
}

# Normalizatsiya
demografics['employment_type'] = demografics['employment_type'].str.lower().map(lambda x: standardized_job_types.get(x, x))

demografics.head()


,cust_id,age,annual_income,employment_length,employment_type,education,marital_status,num_dependents
0,10000,41,61800.0,2.2,full_time,Graduate,Married,2
1,10001,38,28600.0,7.0,full_time,High School,Married,0
2,10002,18,20700.0,0.8,full_time,Bachelor,Single,0
3,10003,27,31400.0,4.8,full_time,Bachelor,Single,0
4,10004,26,24600.0,5.2,full_time,High School,Single,0


In [17]:
geopgrafics_data = pd.read_xml(geo_file)
geopgrafics_data.head()

,id,state,regional_unemployment_rate,regional_median_income,regional_median_rent,housing_price_index,cost_of_living_index,previous_zip_code
0,100000,TN,4.1,53000,1400.0,99.0,73.0,390
1,100001,CA,5.6,75000,1580.0,161.0,130.0,292
2,100002,NY,4.9,72000,1740.0,138.0,118.0,552
3,100003,MA,4.0,81000,1770.0,140.0,116.0,716
4,100004,IL,4.8,65000,1520.0,96.0,86.0,559


In [102]:
geopgrafics_data.isna().sum()

id                            0
state                         0
regional_unemployment_rate    0
regional_median_income        0
regional_median_rent          0
housing_price_index           0
cost_of_living_index          0
previous_zip_code             0
dtype: int64

In [43]:
credit_history=pd.read_parquet(history_file)
credit_history.head()

,customer_number,credit_score,num_credit_accounts,oldest_credit_line_age,oldest_account_age_months,total_credit_limit,num_delinquencies_2yrs,num_inquiries_6mo,recent_inquiry_count,num_public_records,num_collections,account_diversity_index
0,100000,770,8,1.5,18.0,140600.0,0.0,1,1,0,0,0.250
1,100001,733,7,14.6,175.2,90400.0,0.0,1,1,0,0,0.284
2,100002,660,0,4.0,48.0,32400.0,0.0,5,5,0,1,0.050
3,100003,605,14,9.0,108.0,39100.0,NaN,0,0,0,0,0.347
4,100004,721,5,8.7,104.4,47900.0,0.0,2,2,1,0,0.239


In [104]:
selected_credit_history_columns = [
    "customer_number",
    "num_credit_accounts", 
    "oldest_credit_line_age",
    "oldest_account_age_months",
    "total_credit_limit",
    "num_delinquencies_2yrs", 
    "num_inquiries_6mo",
    "recent_inquiry_count", 
    "num_public_records",
    "num_collections",
    "account_diversity_index"
]
credit_history = credit_history[selected_credit_history_columns]

In [105]:
credit_history.isna().sum()

customer_number                0
num_credit_accounts            0
oldest_credit_line_age         0
oldest_account_age_months      0
total_credit_limit             0
num_delinquencies_2yrs       832
num_inquiries_6mo              0
recent_inquiry_count           0
num_public_records             0
num_collections                0
account_diversity_index        0
dtype: int64

In [106]:
credit_history['num_delinquencies_2yrs'] = credit_history['num_delinquencies_2yrs'].fillna(0)


In [107]:
credit_history.isna().sum()

customer_number              0
num_credit_accounts          0
oldest_credit_line_age       0
oldest_account_age_months    0
total_credit_limit           0
num_delinquencies_2yrs       0
num_inquiries_6mo            0
recent_inquiry_count         0
num_public_records           0
num_collections              0
account_diversity_index      0
dtype: int64

# Financial Ratios

In [ ]:
finance_ratios=pd.read_json(financial_file, lines=True)
finance_ratios.head()

In [109]:
finance_ratios = finance_ratios.drop(columns=["debt_to_income_ratio", "revolving_balance"])

for col in finance_ratios.columns:
    finance_ratios[col] = (
        finance_ratios[col]
        .astype(str)
        .str.replace(r"[$,]", "", regex=True)
        .replace("nan", pd.NA)
        .astype(float)
    )

In [44]:
app_data= pd.read_csv(app_file)
app_data = app_data.drop(columns=['random_noise_1', 'application_id'], errors='ignore')
app_data.head()

,customer_ref,application_hour,application_day_of_week,account_open_year,preferred_contact,referral_code,account_status_code,num_login_sessions,num_customer_service_calls,has_mobile_app,paperless_billing
0,100000,21,0,2018,Email,REF0000,ACT-1,12,2,0,0
1,100001,9,6,2021,Phone,REF0000,ACT-1,10,4,0,1
2,100002,10,4,2019,Phone,REF0000,ACT-1,4,3,1,1
3,100003,3,4,2011,Email,REF0000,ACT-2,7,0,1,0
4,100004,9,1,2021,Email,REF0000,ACT-1,6,0,1,0


## Encoding character columns

In [111]:
from sklearn.preprocessing import LabelEncoder

### App-Data

In [112]:
CURRENT_YEAR = 2025
app_data['account_tenure'] = CURRENT_YEAR - app_data['account_open_year']
app_data = app_data.drop(columns=['account_open_year'])

app_data['is_referred'] = (app_data['referral_code'] != 'REF0000').astype(int)
app_data = app_data.drop(columns=['referral_code'])

app_data['preferred_contact'] = app_data['preferred_contact'].replace({'Mail': 'Email'})
app_data['preferred_contact'] = app_data['preferred_contact'].map({'Email': 1, 'Phone': 0})

status_default_rate = app_data.groupby('account_status_code')['default'].mean().sort_values()
status_mapping = {status: idx for idx, status in enumerate(status_default_rate.index)}
app_data['account_status_code'] = app_data['account_status_code'].map(status_mapping)
app_data.head()

,customer_ref,application_hour,application_day_of_week,preferred_contact,account_status_code,num_login_sessions,num_customer_service_calls,has_mobile_app,paperless_billing,default,account_tenure,is_referred
0,10000,5,6,1,3,13,2,1,1,0,12,0
1,10001,4,2,0,1,6,1,1,1,1,10,0
2,10002,10,3,0,1,1,2,1,0,0,5,0
3,10003,7,5,1,4,4,1,1,1,0,15,0
4,10004,1,2,1,1,6,2,1,0,0,5,0


### Demographics

In [113]:

demografics_label_encoders = {}  
demografics_label_columns=['employment_type', 'education', 'marital_status']
for col in demografics_label_columns:
    le = LabelEncoder()
    demografics[col] = le.fit_transform(demografics[col])
    demografics_label_encoders[col] = le  


In [114]:
demografics.head()

,cust_id,age,annual_income,employment_length,employment_type,education,marital_status,num_dependents
0,10000,41,61800.0,2.2,1,2,1,2
1,10001,38,28600.0,7.0,1,3,1,0
2,10002,18,20700.0,0.8,1,1,2,0
3,10003,27,31400.0,4.8,1,1,2,0
4,10004,26,24600.0,5.2,1,3,2,0


### geographics

In [115]:
geographics_label_encoder=LabelEncoder()
geopgrafics_data['state']=geographics_label_encoder.fit_transform(geopgrafics_data['state'])
geopgrafics_data.head()

,id,state,regional_unemployment_rate,regional_median_income,regional_median_rent,housing_price_index,cost_of_living_index,previous_zip_code
0,10000,13,4.8,56000,1380.0,91.0,73.0,451
1,10001,14,4.4,61000,1510.0,92.0,87.0,537
2,10002,17,3.9,74000,1920.0,125.0,103.0,679
3,10003,1,5.8,75000,1690.0,158.0,121.0,719
4,10004,18,5.8,78000,1700.0,152.0,127.0,933


### Loan details

In [116]:
loan_label_encoders = {}  
loan_label_columns=['loan_type', 'loan_purpose', 'origination_channel','marketing_campaign']
for col in loan_label_columns:
    le = LabelEncoder()
    loan_details[col] = le.fit_transform(loan_details[col])
    loan_label_encoders[col] = le 
loan_details.head() 

,customer_id,loan_type,loan_amount,loan_term,interest_rate,loan_purpose,loan_to_value_ratio,origination_channel,loan_officer_id,marketing_campaign
0,10000,2,17700.0,36,12.50,0,0.000,2,1045,22
1,10001,1,114000.0,180,6.83,6,0.774,0,1011,1
2,10002,2,9300.0,36,13.99,3,0.000,3,1084,10
3,10003,2,8700.0,48,13.26,4,0.000,3,1048,0
4,10004,2,7200.0,24,10.77,0,0.000,0,1055,18


### training model

In [117]:
final_df = loan_details.merge(demografics, left_on='customer_id', right_on="cust_id", how='outer').merge(geopgrafics_data,left_on="customer_id",right_on="id", how='outer').merge(credit_history,left_on="customer_id",right_on="customer_number",how="outer")
final_df=final_df.merge(app_data,left_on="customer_id",right_on="customer_ref",how="outer").merge(finance_ratios,left_on="customer_id",right_on="cust_num",how="outer")
final_df.columns

Index(['customer_id', 'loan_type', 'loan_amount', 'loan_term', 'interest_rate',
       'loan_purpose', 'loan_to_value_ratio', 'origination_channel',
       'loan_officer_id', 'marketing_campaign', 'cust_id', 'age',
       'annual_income', 'employment_length', 'employment_type', 'education',
       'marital_status', 'num_dependents', 'id', 'state',
       'regional_unemployment_rate', 'regional_median_income',
       'regional_median_rent', 'housing_price_index', 'cost_of_living_index',
       'previous_zip_code', 'customer_number', 'num_credit_accounts',
       'oldest_credit_line_age', 'oldest_account_age_months',
       'total_credit_limit', 'num_delinquencies_2yrs', 'num_inquiries_6mo',
       'recent_inquiry_count', 'num_public_records', 'num_collections',
       'account_diversity_index', 'customer_ref', 'application_hour',
       'application_day_of_week', 'preferred_contact', 'account_status_code',
       'num_login_sessions', 'num_customer_service_calls', 'has_mobile_app',
    

In [118]:
y=app_data['default']
y


0        0
1        1
2        0
3        0
4        0
        ..
89994    0
89995    0
89996    1
89997    0
89998    0
Name: default, Length: 89999, dtype: int64

In [119]:
X = final_df.drop([ 'customer_id', 'cust_id', 'id','customer_number','customer_ref','cust_num','default','previous_zip_code','state','marketing_campaign','loan_officer_id'], axis=1)

In [120]:
X.to_csv("../merge/merged_x.csv",index=False)
y.to_csv("../merge/merged_y.csv",index=False)